In [1]:
import glob, os
import fnmatch
import sys
import astropy
import json
import parsnip
import pandas as pd
from matplotlib import pyplot as plt
import h5py
import numpy as np
from scipy.optimize import curve_fit
import lcdata
from astropy.table import Table
import csv
from tika import parser

In [2]:
def parse_jsons(photometry_path, object_name):
    '''
    Finds object ID, ra, and dec for older data from a json file.
    '''
    jsons = ""
    for file in os.listdir(f'{photometry_path}{object_name}/'):
        # print(file)
        if file.endswith(".json"):
            # print('hi')
            f = open(f'{photometry_path}{object_name}/{file}')
            data = json.load(f)
            print(data.keys())
            # print(data)
            id = data["ObjectID"]
            # id = 11
            ra = data['RA-OBJECT']
            dec = data['DEC-OBJECT']
            jsons = True
            # print (id, ra, dec)
    if not jsons:
        print("could not find json file for "+object_name+", not creating lcdata for it")
        return "", "", ""
        
    return id, ra, dec

In [3]:
def parse_pdfs(photometry_path, object_name):
    '''
    Finds redshift from pdf files for older data. 
    '''
    text = ''
    for file in os.listdir(f'{photometry_path}{object_name}/'):
        if file.endswith(".pdf"):
            # print(file)
            pdf = parser.from_file(f'{photometry_path}{object_name}/{file}')
            text = pdf['content']
            
    if not text:
        print(f'Error with finding pdf file/redshift for {object_name}, setting redshift to 0')
        return 0
            # print(text)

    zs = ""
    for i in range(len(text)):
        if text[i].startswith("T"):
            # find the redshift for the dataset
            for j in range(i, i+50):
                if text[j:j+8].startswith('| 0.'):
                    substring = text[j+2:j+9]
                    if "\n" in substring:
                        substring = substring.replace("\n", "")  # Remove newline character if present
                    zs += substring
            break

        if text[i].startswith("A"):
            # find the redshift for the dataset
            for j in range(i, i+50):
                if text[j:j+8].startswith('| 0.'):
                    substring = text[j+2:j+9]
                    if "\n" in substring:
                        substring = substring.replace("\n", "")  # Remove newline character if present
                    zs += substring
            break

    if not zs:
        #print(f'Could not find redshift for {object_name}; setting it to 0')
        zs = "0"
        
    # print(f'Redshift is {zs}')
    return zs


In [4]:
def make_lctable(photometry_path, object_name):
    '''
    Looks at csv file for given object and creates a light curve from it. 
    '''
    lightcurveTable = Table(names=('time(mjd)', 'flux', 'fluxerr', 'mag', 'magerr', 'band'), dtype=('f', 'f', 'f', 'f', 'f','S'))

    for file in os.listdir(f'{photometry_path}{object_name}/'):
        if file.endswith(".csv"):
            csvFile = open(f'{photometry_path}{object_name}/{file}')
            reader = csv.DictReader(csvFile)
            for row in reader:
                if row['alert'] == 'True':
                    time = row['mjd']
                    mag = float(row['mag'])
                    magerr = float(row['magerr'])
                    flux = 10**((22.5-mag)/2.5)
                    fluxerr = flux * np.log(10) / 2.5 * magerr
                    band = 'lsst' + row['filter']
                    lightcurveTable.add_row((time, flux, fluxerr, mag, magerr, band))
                    
    return lightcurveTable

In [5]:
#combine the two tables into lcdata
def combine_tables(lightcurveTable, id, ra, dec, zs, type='nan'):
    
    light_curve = Table({
         'time': list(lightcurveTable['time(mjd)']),
         'flux': list(lightcurveTable['flux']),
         'fluxerr': list(lightcurveTable['fluxerr']),
         'mag': list(lightcurveTable['mag']),
         'magerr': list(lightcurveTable['magerr']),
         'band': list(lightcurveTable['band']),})

    # print(light_curve, '\n')

    light_curve.meta = {
        'object_id': id,
        'ra': ra,
        'dec': dec,
        'type': type,
        'redshift': zs}
    
    dataset = lcdata.from_light_curves([light_curve])
    
    return dataset

# print(light_curve.meta)

In [6]:
def lcformat_old_data(photometry_path, object_name, output_dir, type='nan'):
    id, ra, dec = parse_jsons(photometry_path, object_name)
    
    # if not id:
    #     print(f'Incorrectly recognized {object_name} as an object. Skipping this one')
    
    if id:
        zs = parse_pdfs(photometry_path, object_name)

        lctable = make_lctable(photometry_path, object_name)


        lcdataset = combine_tables(lctable, id, ra, dec, zs, type)

        output_path = f'{output_dir}{object_name}.hdf5'
    
        directory = os.path.dirname(os.path.abspath(output_path))
        os.makedirs(directory, exist_ok=True)

        if os.path.exists(output_path):
            exists.append(output_path)
            #print(f"File '{output_path}' already exists. Skipping dataset writing.")
        else:
            lcdataset.write_hdf5(output_path)
            #print(f"Dataset ({len(lcdataset)} objects) has been written into {output_path}")
    
    

In [7]:
def fit_mags(photometry_path, object_name):
    '''
    Looks at csv file for given object and returns array of its mags and mag errors
    '''
    maglist = []
    magerrlist = []
    for file in os.listdir(f'{photometry_path}{object_name}/'):
        if file.endswith(".csv"):
            csvFile = open(f'{photometry_path}{object_name}/{file}')
            reader = csv.DictReader(csvFile)
            for row in reader:
                if 'alert' in row:
                    if row['alert'] == 'True':
                        mag = float(row['mag'])
                        magerr = float(row['magerr'])
                        maglist.append(mag)
                        magerrlist.append(magerr)
                    
    return maglist, magerrlist

In [20]:
#get all old data in lcdata format
photometry_path = '/global/cfs/cdirs/m4237/desirt/2021_data/aperture_photometry/LCData_Legacy/'
output_dir = '/global/cfs/cdirs/m4237/desirt/2021_data/lcdata/'

magstotal = []
magerrstotal = []
for objectname in os.listdir(photometry_path):
    if os.path.isdir(f'{photometry_path}{objectname}'):
        # parse_jsons(photometry_path, objectname)
        # lcformat_old_data(photometry_path, objectname, output_dir)
        maglist, magerrlist = fit_mags(photometry_path, objectname)
        magstotal.append(maglist)
        magerrstotal.append(magerrstotal)
        

In [8]:
#section for new data
import os
# os.listdir('/global/cfs/cdirs/m4237/gesa/DCDE4/ObjectDir/Legacy/A202303191446357p072803')
new_path = '/global/cfs/cdirs/m4237/gesa/DCDE/ObjectDir/Legacy/'
# print(os.listdir(new_path))
# object_name = 'T202304091111386m063203'
object_name = 'A202302110338163m160947'

In [9]:
#new data
def new_files_metadata(new_path, object_name):
    directory_path = f'{new_path}{object_name}/'
    id_name = str(object_name)
    ra = ""
    dec = ""
    lightcurveTable = Table(names=('time(mjd)', 'flux', 'fluxerr', 'mag', 'magerr', 'band'), dtype=('f', 'f', 'f', 'f', 'f','S'))

    if not os.path.isdir(directory_path):
        # print(f'{directory_path} is not a directory')
        return None, None, None
    if not os.path.exists(f'{new_path}{object_name}/alertUltra.csv'):
        print(f'No alertUltra.csv file for {object_name}.')
        # print(os.listdir(f'{new_path}{object_name}/'))
        return None, None, None

    csvFile = open(f'{new_path}{object_name}/alertUltra.csv')
    reader = csv.DictReader(csvFile)


    for row in reader:
        # print(row)
        ra = row['legacyGalRa']
        dec = row['legacyGalDec']
        # if 'ObjectID' in row:
        #     id_name = row['ObjectID']
            # print(id_name)
        # else:
        #     id_name = str(object_name)
                
    return id_name, ra, dec


In [10]:
def new_files_lctable(new_path, object_name):
    lightcurveTable = Table(names=('time(mjd)', 'flux', 'fluxerr', 'mag', 'magerr', 'band'), 
                            dtype=('f', 'f', 'f', 'f', 'f','S'))
    if os.path.exists(f'{new_path}{object_name}/alertUltra.csv'):
        csvFile = open(f'{new_path}{object_name}/alertUltra.csv')
        reader = csv.DictReader(csvFile)

        for row in reader:
            if 'AlertID' in row:
                if row['AlertID'] and row['MAG_APERc']:
                    time = row['MJD_OBS_SCI']
                    # print(row['MAG_APERc'])
                    mag = float(row['MAG_APERc'])
                    magerr = float(row['MAGERR_APER'])
                    flux = 10**((22.5-mag)/2.5)
                    fluxerr = flux * np.log(10) / 2.5 * magerr
                    filename = row['REFNAME']
                    long_string = filename.split("_")[4]
                    band = 'lsst' + long_string
                    lightcurveTable.add_row((time, flux, fluxerr, mag, magerr, band))

        return lightcurveTable
    
    else:
        print(object_name)


In [11]:
def find_redshift(target_id, csv_file_path='decam_desi_crossmatch.csv'):
    '''
    Looks to see if redshift present in the redshift matched decam_desi_crossmatch.csv file
    '''
    zs = ""
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # Skip the header row
        for row in csv_reader:
            # print(row)
            if row[1] == target_id:
                # print(row)
                if row[4]:
                    zs = row[4]
                    #print(f'Found z={zs} for {object_name}')
                    # print(zs)
        # If no match is found
        if not zs:
            # print(f'Could not find redshift for {object_name}; setting it to 0')
            zs = "0"
        return zs

In [13]:
find_redshift('A202302110338163m160947')

'0'

In [12]:
no_id = []
exists= []
looped = []
written = []
ids = []

In [13]:
def convert_lcdata_new(new_path, object_name, output_dir):
    id, ra, dec = new_files_metadata(new_path, object_name)
    looped.append(object_name)
    ids.append(id)
    # print(id, ra, dec)
    if not id:
        no_id.append(object_name)
#         print(f'Incorrectly recognized {object_name} as an object, or no ID found (non-detection). Skipping this one')
    
    if id:
        lctable = new_files_lctable(new_path, object_name)
        zs = find_redshift(str(id))
        # if zs:
        #     print(f'object {id} has z={zs}')
        
        lcdataset = combine_tables(lctable, id, ra, dec, zs, type)

        output_path = f'{output_dir}{object_name}.hdf5'
    
        # directory = os.path.dirname(os.path.abspath(output_path))
        # os.makedirs(directory, exist_ok=True)

        if os.path.exists(output_path):
            #print(f"File '{output_path}' already exists. Skipping dataset writing.")
            exists.append(output_path)
        else:
            lcdataset.write_hdf5(output_path)
            written.append(output_path)
            #print(f"Dataset ({len(lcdataset)} objects) was written into {output_path} w/ z={zs}")
    
    

In [78]:
new_path = '/global/cfs/cdirs/m4237/gesa/DCDE/ObjectDir/Legacy/'
obj_list = []
ra_list = []
for object_name in os.listdir(new_path):
    obj_list.append(object_name)

In [79]:


with tqdm(total=len(obj_list), desc='Progress', unit='file') as pbar:
    for object_name in obj_list:
        convert_lcdata_new(new_path, object_name, 
                           '/global/cfs/cdirs/m4237/desirt/new_data/DCDE/')
        pbar.update(1)

Progress: 100%|██████████| 1005/1005 [00:22<00:00, 44.48file/s]


In [81]:
print(len(exists), len(looped), len(no_id), len(written), len(ids))
# print(ids)

175 1005 24 806 1005


In [67]:
obj_list = []
for obj in os.listdir('/global/cfs/cdirs/m4237/desirt/new_data/DCDE/'):
    obj_list.append(obj)
print(len(obj_list))

175


In [82]:
no_id = []
exists= []
looped = []
written = []
ids = []

new_path = '/global/cfs/cdirs/m4237/gesa/DCDE2/ObjectDir/Legacy/'
obj_list = []

for object_name in os.listdir(new_path):
    obj_list.append(object_name)

for object_name in obj_list:
    convert_lcdata_new(new_path, object_name, '/global/cfs/cdirs/m4237/desirt/new_data/DCDE2/')
    
print(len(exists), len(looped), len(no_id), len(written), len(ids))

No alertUltra.csv file for T202207041235453p075324.
No alertUltra.csv file for A202207011259022p064616.
No alertUltra.csv file for A202207011500532p065000.
No alertUltra.csv file for A202207011425499m011445.
No alertUltra.csv file for A202206282123268p050749.
No alertUltra.csv file for A202207011413372m000630.
No alertUltra.csv file for T202207072257222p005232.
No alertUltra.csv file for T202206282115439p044616.
No alertUltra.csv file for T202206281507193p070252.
No alertUltra.csv file for T202206281443596p073326.
No alertUltra.csv file for A202207072256121m004511.
No alertUltra.csv file for A202207011454476p055337.
No alertUltra.csv file for T202207012113018p052516.
No alertUltra.csv file for T202207072254254m002223.
No alertUltra.csv file for T202207041517577p072340.
No alertUltra.csv file for A202207011257504p055631.
No alertUltra.csv file for A202207011253179p064427.
No alertUltra.csv file for A202206282125412p063603.
No alertUltra.csv file for A202206282118233p064223.
No alertUltr

In [83]:
no_id = []
exists= []
looped = []
written = []
ids = []

new_path = '/global/cfs/cdirs/m4237/gesa/DCDE3/ObjectDir/Legacy_new/'
obj_list = []

for object_name in os.listdir(new_path):
    obj_list.append(object_name)

for object_name in obj_list:
    convert_lcdata_new(new_path, object_name, '/global/cfs/cdirs/m4237/desirt/new_data/DCDE3/')
    
print(len(exists), len(looped), len(no_id), len(written), len(ids))

187 1125 39 899 1125


In [15]:
new_path = '/global/cfs/cdirs/m4237/gesa/DCDE3/ObjectDir/Legacy_new/'
convert_lcdata_new(new_path, 'A202210060344377m154246', '/global/cfs/cdirs/m4237/desirt/new_data/DCDE3/')

In [84]:
no_id = []
exists= []
looped = []
written = []
ids = []

new_path = '/global/cfs/cdirs/m4237/gesa/DCDE4/ObjectDir/Legacy/'
obj_list = []

for object_name in os.listdir(new_path):
    obj_list.append(object_name)

with tqdm(total=len(obj_list), desc='Progress', unit='file') as pbar:
    for object_name in obj_list:
        convert_lcdata_new(new_path, object_name, 
                           '/global/cfs/cdirs/m4237/desirt/new_data/DCDE4/')
        pbar.update(1)
print(len(exists), len(looped), len(no_id), len(written), len(ids))

Progress:   1%|          | 8/927 [00:00<00:56, 16.22file/s]

No alertUltra.csv file for A202306081508122p063439.
No alertUltra.csv file for A202306141454476p055337.


Progress:   3%|▎         | 24/927 [00:01<00:44, 20.14file/s]

No alertUltra.csv file for A202304241258157p055756.
No alertUltra.csv file for A202306141424172m001215.
No alertUltra.csv file for A202306051515060p082417.
No alertUltra.csv file for T202305152208525m044142.
No alertUltra.csv file for A202304121331268p162834.


Progress:   4%|▍         | 35/927 [00:02<00:49, 18.15file/s]

...

No alertUltra.csv file for A202304271504341p073611.
No alertUltra.csv file for A202305152224113m025449.
No alertUltra.csv file for A202306141456172p053916.
No alertUltra.csv file for T202305121330323p184508.


Progress:  98%|█████████▊| 911/927 [00:29<00:00, 49.05file/s]

No alertUltra.csv file for A202304121251224p070010.
No alertUltra.csv file for A202304121418196p001417.
No alertUltra.csv file for A202304271049243m064020.
No alertUltra.csv file for A202304241301428p071530.


Progress: 100%|██████████| 927/927 [00:29<00:00, 31.30file/s]

No alertUltra.csv file for A202306141503054p080002.
No alertUltra.csv file for T202304091111386m063203.
195 927 285 447 927
